In [29]:
!pip install pandas numpy scikit-learn torch

In [30]:
# check torch gpu
import torch

device = None

print("<!> Selecting GPU as main device <!>")
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("<!> Using: " + torch.cuda.get_device_name(0) + "<!>")
else:
    print("<!> No GPU available <!>")
    exit(0)

<!> Selecting GPU as main device <!>
<!> Using: NVIDIA GeForce GTX 1050<!>


In [31]:
import pandas as pd

print("<!> Loading train dataframe <!>")
train_df = pd.read_csv('/root/test/train_transactions.csv')

<!> Loading train dataframe <!>


In [32]:
import numpy as np

def create_sequences(data, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(sequences), np.array(targets)

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

print("<!> Generating series <!>")
# Extract demand for product 2
train_series = train_df[str(2)].values.reshape(-1, 1)

print("<!> Normalizing data <!>")
# Normalize data
scaler = MinMaxScaler()
train_series = scaler.fit_transform(train_series)

print("<!> Generating sequences data for 30 days <!>")
SEQ_LENGTH = 30 
X_train, y_train = create_sequences(train_series, SEQ_LENGTH)

print("<!> Generating PyTorch tensors <!>")
# Convert to PyTorch tensors and move to device
X_train, y_train = torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device)

# Create Dataset class
class DemandDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = DemandDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


<!> Generating series <!>
<!> Normalizing data <!>
<!> Generating sequences data for 30 days <!>
<!> Generating PyTorch tensors <!>


In [34]:
# Define Bidirectional LSTM Model with Dropout
class BiLSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate=0.2):
        super(BiLSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.fc1 = nn.Linear(hidden_size * 2, 64)  # Bidirectional => hidden_size * 2
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  
        last_step_output = lstm_out[:, -1, :]  # Take last time step
        x = self.fc1(last_step_output)
        x = self.relu(x)
        return self.fc2(x)  # Final output layer

# Model setup
INPUT_SIZE = 1  # Single feature (demand)
HIDDEN_SIZE = 128
NUM_LAYERS = 3
OUTPUT_SIZE = 1
DROPOUT_RATE = 0.2

In [35]:
print("<!> Training <!>")

model = BiLSTMPredictor(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT_RATE).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 50

def train_model():
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to device
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

train_model()

print("<!> Done <!>")

<!> Training <!>


Epoch 1, Loss: 0.0471
Epoch 2, Loss: 0.0148
Epoch 3, Loss: 0.0130
Epoch 4, Loss: 0.0144
Epoch 5, Loss: 0.0129
Epoch 6, Loss: 0.0135
Epoch 7, Loss: 0.0137
Epoch 8, Loss: 0.0128
Epoch 9, Loss: 0.0130
Epoch 10, Loss: 0.0128
Epoch 11, Loss: 0.0130
Epoch 12, Loss: 0.0125
Epoch 13, Loss: 0.0127
Epoch 14, Loss: 0.0119
Epoch 15, Loss: 0.0084
Epoch 16, Loss: 0.0073
Epoch 17, Loss: 0.0051
Epoch 18, Loss: 0.0034
Epoch 19, Loss: 0.0028
Epoch 20, Loss: 0.0029
Epoch 21, Loss: 0.0028
Epoch 22, Loss: 0.0027
Epoch 23, Loss: 0.0026
Epoch 24, Loss: 0.0031
Epoch 25, Loss: 0.0022
Epoch 26, Loss: 0.0021
Epoch 27, Loss: 0.0022
Epoch 28, Loss: 0.0024
Epoch 29, Loss: 0.0025
Epoch 30, Loss: 0.0021
Epoch 31, Loss: 0.0022
Epoch 32, Loss: 0.0022
Epoch 33, Loss: 0.0030
Epoch 34, Loss: 0.0021
Epoch 35, Loss: 0.0021
Epoch 36, Loss: 0.0021
Epoch 37, Loss: 0.0020
Epoch 38, Loss: 0.0021
Epoch 39, Loss: 0.0020
Epoch 40, Loss: 0.0019
Epoch 41, Loss: 0.0020
Epoch 42, Loss: 0.0025
Epoch 43, Loss: 0.0020
Epoch 44, Loss: 0.00

In [36]:
print("<!> Saving model <!>")
torch.save(model.state_dict(), "/root/test/lstm_trained_data.pth")

<!> Saving model <!>
